In [21]:
# repositório do yolov4 no tensorflow
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git
# reositório vant
!git clone https://github.com/vitoryeso/ECT-proj-cnn-vant.git

/content
Cloning into 'tensorflow-yolov4-tflite'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 704 (delta 6), reused 16 (delta 5), pack-reused 682
Receiving objects: 100% (704/704), 176.13 MiB | 13.30 MiB/s, done.
Resolving deltas: 100% (327/327), done.
Cloning into 'ECT-proj-cnn-vant'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 289 (delta 81), reused 278 (delta 75), pack-reused 0
Receiving objects: 100% (289/289), 3.50 MiB | 3.23 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [22]:
%cd tensorflow-yolov4-tflite/

/content/tensorflow-yolov4-tflite


In [23]:
import tensorflow as tf
from core.yolov4 import YOLO, decode, filter_boxes
import core.utils as utils
from core.config import cfg

In [24]:
def load_config(tiny):
    if tiny:
        STRIDES = np.array(cfg.YOLO.STRIDES_TINY)
        ANCHORS = get_anchors(cfg.YOLO.ANCHORS_TINY, tiny)
        XYSCALE = cfg.YOLO.XYSCALE_TINY
    else:
        STRIDES = np.array(cfg.YOLO.STRIDES)
        ANCHORS = utils.get_anchors(cfg.YOLO.ANCHORS, tiny)
        XYSCALE = cfg.YOLO.XYSCALE
    NUM_CLASS = len(utils.read_class_names(cfg.YOLO.CLASSES))

    return STRIDES, ANCHORS, NUM_CLASS, XYSCALE

In [25]:
input_size = 416
tiny = False
score_thresh = 0.5

weights = "" #path pros pesos
output = "" #path + nome pra salvar o modelo
cfg.YOLO.CLASSES = "/content/ECT-proj-cnn-vant/dataset/classes.names"

In [28]:
strides, anchors, num_class, xyscale = load_config(tiny)
input_layer = tf.keras.layers.Input([input_size, input_size, 3])
feature_maps = YOLO(input_layer, num_class, "yolov4", tiny)

bbox_tensors = []
prob_tensors = []

In [ ]:
if tiny:
    for i, feature_map in enumerate(feature_maps):
        if i == 0:
            output_tensors = decode(feature_map, input_size // 16, num_class, strides, anchors, i, xyscale, "tf")
        else:
            output_tensors = decode(feature_map, input_size // 32, num_class, strides, anchors, i, xyscale, "tf")
        bbox_tensors.append(output_tensors[0])
        prob_tensors.append(output_tensors[1])
else:
    for i, feature_maps in enumerate(feature_maps):
        if i == 0:
            output_tensors = decode(feature_map, input_size // 8, num_class, strides, anchors, i , xy_scale, "tf")
        elif i == 1:
            output_tensors = decode(feature_map, input_size // 16, num_class, strides, anchors, i , xy_scale, "tf")
        else:
            output_tensors = decode(feature_map, input_size // 32, num_class, strides, anchors, i , xy_scale, "tf")
        bbox_tensors.append(output_tensors[0])
        prob_tensors.append(output_tensors[1])

pred_bbox = tf.concat(bbox_tensors, axis=1)
pred_prob = tf.concat(bbox_tensors, axis=1)

boxes, pred_confidence = filter_boxes(pred_bbox, pred_prob, score_threshold=score_thresh, input_shape=tf.constant([input_size]*2))
pred = tf.concat([boxes, pred_confidence], axis=-1)

In [ ]:
model = tf.keras.Model(input_layer, pred)


In [ ]:
utils.load_weights(model, weights, "yolov4", tiny)

In [ ]:
model.summary()

# se quiser salvar o modelo
# model.save("name")